<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# =========================================================
# HW2（本地分析版）：不使用 Gemini / OpenAI
# - Google Sheet I/O + AA 分攤 + Gradio
# - 本地規則產生「AI 分析」並寫回 AI_Analysis + Summary
# =========================================================

# 0) 安裝套件
!pip -q install --upgrade gspread pandas google-auth google-auth-oauthlib pytz gradio

# 1) 基本設定
SHEET_URL = "https://docs.google.com/spreadsheets/d/1zgd1GM9-Vi0JlwacPmbZF0Ci45QSX2t0EIZY1fmzfFE/edit?gid=0#gid=0"
DATA_SHEET_NAME = "消費紀錄"
SUMMARY_SHEET_NAME = "Summary"
AI_SHEET_NAME = "AI_Analysis"
TIMEZONE = "Asia/Taipei"
DEBUG = True

# 2) Google 驗證
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# 3) 依賴
import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timedelta, date
import gradio as gr
import traceback

# 4) 共用函式
def debug_text(e):
    return f"{type(e).__name__}: {e}\n\n{traceback.format_exc()}" if DEBUG else f"{type(e).__name__}: {e}"

def open_spreadsheet():
    if "docs.google.com/spreadsheets/d/" not in SHEET_URL:
        raise ValueError("SHEET_URL 格式不正確。")
    return gc.open_by_url(SHEET_URL)

def get_ws(ss, title):
    try:
        return ss.worksheet(title)
    except gspread.WorksheetNotFound:
        return None

def ensure_sheet(ss, title, rows=200, cols=20):
    ws = get_ws(ss, title)
    if ws is None:
        ws = ss.add_worksheet(title=title, rows=rows, cols=cols)
    return ws

def parse_date_safe(x):
    if pd.isna(x) or str(x).strip() == "":
        return pd.NaT
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def to_number(v):
    s = str(v).replace(",", "").strip()
    try:
        return float(s)
    except Exception:
        return np.nan

def load_data():
    ss = open_spreadsheet()
    ws = get_ws(ss, DATA_SHEET_NAME)
    if ws is None:
        raise RuntimeError(f"找不到工作表『{DATA_SHEET_NAME}』。")

    values = ws.get_all_values()
    if not values:
        raise RuntimeError(f"『{DATA_SHEET_NAME}』是空的；第 1 列需為：date, category, item, amount, payer")

    headers = [h.strip().lower() for h in values[0]]
    df = pd.DataFrame(values[1:], columns=headers)
    required = ["date", "category", "item", "amount", "payer"]
    miss = [c for c in required if c not in df.columns]
    if miss:
        raise RuntimeError(f"缺少必要欄位：{miss}；第 1 列需為 {required}")

    df["date"] = df["date"].apply(parse_date_safe)
    df["amount"] = df["amount"].apply(to_number)
    df = df.dropna(subset=["date","category","item","amount","payer"]).reset_index(drop=True)
    if df.empty:
        raise RuntimeError("有效資料為空（請檢查日期/金額/空白列）")
    return ss, df

def compute_summary(df):
    total_amount = float(df["amount"].sum())
    by_category = (
        df.groupby("category", dropna=False)["amount"]
          .sum().reset_index().sort_values("amount", ascending=False)
    )

    members = sorted(df["payer"].dropna().astype(str).unique().tolist())
    n = len(members)
    equal_share = total_amount / n if n>0 else 0.0
    paid_by = (df.groupby("payer", dropna=False)["amount"]
                 .sum().reindex(members, fill_value=0).reset_index())
    paid_by.columns = ["payer","paid"]
    aa_summary = paid_by.copy()
    aa_summary["equal_share"] = round(equal_share,2)
    aa_summary["balance"] = (aa_summary["paid"] - equal_share).round(2)

    # 轉帳建議
    pos = aa_summary[aa_summary["balance"]>0][["payer","balance"]].copy().sort_values("balance", ascending=False).reset_index(drop=True)
    neg = aa_summary[aa_summary["balance"]<0][["payer","balance"]].copy().sort_values("balance").reset_index(drop=True)
    transfers=[]; i=j=0
    while i<len(neg) and j<len(pos):
        owe_name, owe_amt = neg.loc[i,"payer"], -float(neg.loc[i,"balance"])
        rec_name, rec_amt = pos.loc[j,"payer"],  float(pos.loc[j,"balance"])
        pay = round(min(owe_amt, rec_amt), 2)
        if pay>0:
            transfers.append({"from": owe_name, "to": rec_name, "amount": pay})
            neg.loc[i,"balance"] = -(owe_amt - pay)
            pos.loc[j,"balance"] =  (rec_amt - pay)
        if neg.loc[i,"balance"] >= -1e-9: i+=1
        if pos.loc[j,"balance"] <=  1e-9: j+=1
    transfer_df = pd.DataFrame(transfers) if transfers else pd.DataFrame([{"from":"—","to":"—","amount":0}])

    # 本週
    tz = pytz.timezone(TIMEZONE)
    today = datetime.now(tz).date()
    week_start = today - timedelta(days=today.weekday())
    week_end   = week_start + timedelta(days=6)
    df_week = df[(df["date"].dt.date>=week_start) & (df["date"].dt.date<=week_end)]

    if df_week.empty:
        weekly_text = f"本週（{week_start}~{week_end}）尚無消費紀錄。"
    else:
        top_cat = df_week.groupby("category")["amount"].sum().sort_values(ascending=False)
        tip = "" if top_cat.empty else f"最高支出類別「{top_cat.index[0]}」占比 {float(top_cat.iloc[0]/top_cat.sum()*100):.1f}%。"
        high = df_week.sort_values("amount", ascending=False).head(1)
        extra = "" if high.empty or float(high.iloc[0]['amount'])<1000 else f" 最大單筆「{high.iloc[0]['item']}」{int(high.iloc[0]['amount'])}。"
        weekly_text = f"本週（{week_start}~{week_end}）總支出 {float(df_week['amount'].sum()):.0f}。 {tip}{extra}"

    return {
        "df": df, "by_category": by_category, "aa_summary": aa_summary, "transfer_df": transfer_df,
        "total_amount": total_amount, "members": members, "equal_share": equal_share,
        "week_start": week_start, "week_end": week_end, "df_week": df_week, "weekly_text": weekly_text
    }

def write_summary(blocks):
    ss = open_spreadsheet()
    ws = ensure_sheet(ss, SUMMARY_SHEET_NAME)

    def _to_native(v):
        try:
            if pd.isna(v): return ""
        except Exception:
            pass
        if isinstance(v, (np.integer,)): return int(v)
        if isinstance(v, (np.floating,)): return float(v)
        if isinstance(v, (pd.Timestamp, datetime, date)):
            return pd.to_datetime(v).strftime("%Y-%m-%d")
        return str(v)

    def write_block(start_row, start_col, values):
        conv = [[_to_native(x) for x in row] for row in values]
        end_row = start_row + len(conv) - 1
        end_col = start_col + len(conv[0]) - 1
        rng = gspread.utils.rowcol_to_a1(start_row, start_col) + ":" + gspread.utils.rowcol_to_a1(end_row, end_col)
        ws.update(values=conv, range_name=rng)
        return end_row + 2

    overview_rows = [
        ["指標","數值"],
        ["總支出", round(float(blocks["total_amount"]),2)],
        ["成員（AA）", ", ".join(blocks["members"])],
        ["平均分攤(每人)", round(float(blocks["equal_share"]),2)],
        ["週期(本週)", f"{blocks['week_start']} ~ {blocks['week_end']}"],
    ]
    cat_table = [["category","subtotal"]] + [[str(r["category"]), round(float(r["amount"]),2)] for _,r in blocks["by_category"].iterrows()]
    aa_table  = [["payer","paid","equal_share","balance"]] + [
        [str(r["payer"]), round(float(r["paid"]),2), round(float(r["equal_share"]),2), round(float(r["balance"]),2)]
        for _,r in blocks["aa_summary"].iterrows()
    ]
    tr_table  = [["from","to","amount"]] + [[str(r["from"]), str(r["to"]), round(float(r["amount"]),2)] for _,r in blocks["transfer_df"].iterrows()]
    summary_paragraph = [["=== Weekly Summary (Auto) ==="], [blocks["weekly_text"]]]

    ws.clear()
    row = 1
    row = write_block(row, 1, [["=== Overview ==="]]);             row = write_block(row, 1, overview_rows)
    row = write_block(row, 1, [["=== Category Subtotals ==="]]);   row = write_block(row, 1, cat_table)
    row = write_block(row, 1, [["=== AA Split ==="]]);             row = write_block(row, 1, aa_table)
    row = write_block(row, 1, [["=== Suggested Transfers ==="]]);  row = write_block(row, 1, tr_table)
    row = write_block(row, 1, summary_paragraph)
    return "已寫回 Summary ✅"

# 5) 本地「AI」分析（規則引擎）
def local_ai_analysis(df_week, week_start, week_end):
    """
    規則要點：
    - 找出最高類別、最高單筆
    - 外食/飲料/娛樂占比過高提出建議
    - 連續多天外食/交通偏高/書籍學習加分
    - 平日/週末消費差異
    - 大於 1000 的單筆給提醒
    - 以 150~220 字左右輸出
    """
    if df_week.empty:
        return "local", f"本週（{week_start}~{week_end}）尚無消費紀錄，無法生成分析。"

    s = []
    total = float(df_week["amount"].sum())
    cat = df_week.groupby("category")["amount"].sum().sort_values(ascending=False)
    top_cat = cat.index[0] if not cat.empty else "—"
    top_cat_ratio = (float(cat.iloc[0])/total*100) if total>0 and not cat.empty else 0.0

    s.append(f"本週（{week_start}~{week_end}）總支出約 {int(total)} 元，最高支出類別為「{top_cat}」，約占 {top_cat_ratio:.1f}%。")

    # 單筆最大
    hi = df_week.sort_values("amount", ascending=False).iloc[0]
    if float(hi["amount"]) >= 1000:
        s.append(f"單筆較高支出為「{hi['item']}」{int(hi['amount'])} 元，可檢視是否必要或可分期/延後。")

    # 外食/飲料/娛樂占比
    def ratio_of(keys):
        sub = df_week[df_week["category"].isin(keys)]["amount"].sum()
        return (float(sub)/total*100) if total>0 else 0.0
    eat_ratio = ratio_of(["餐飲","外食","飲料"])
    ent_ratio = ratio_of(["娛樂","遊戲","影音"])

    if eat_ratio > 35:
        s.append(f"飲食相關支出占比 {eat_ratio:.1f}% 偏高，建議一週至少 {int((eat_ratio-30)//5+1)} 次自煮或調整外食單價。")
    if ent_ratio > 15:
        s.append(f"娛樂支出占比 {ent_ratio:.1f}%，可設定每週娛樂上限，優先選擇免費/低價活動。")

    # 交通偏高
    trans_ratio = ratio_of(["交通"])
    if trans_ratio > 20:
        s.append(f"交通占比 {trans_ratio:.1f}% 較高，可善用月票/共乘/提前購票。")

    # 書籍/學習類
    learn_ratio = ratio_of(["書籍","學習","課程"])
    if learn_ratio >= 10:
        s.append(f"投資學習占比 {learn_ratio:.1f}%，屬正向支出，建議持續但掌握預算。")

    # 連續外食天數
    df_week["weekday"] = df_week["date"].dt.weekday
    day_eat = (df_week[df_week["category"].isin(["餐飲","外食","飲料"])]
               .groupby(df_week["date"].dt.date)["amount"].sum())
    long_streak = 0; curr=0
    for d in sorted(set(df_week["date"].dt.date)):
        if d in day_eat.index:
            curr += 1
            long_streak = max(long_streak, curr)
        else:
            curr = 0
    if long_streak >= 4:
        s.append(f"本週連續 {long_streak} 天有外食/飲料，嘗試安排『自煮日』或準備便當。")

    # 平日 vs 週末
    weekday_spend = float(df_week[df_week["date"].dt.weekday<=4]["amount"].sum())
    weekend_spend = float(df_week[df_week["date"].dt.weekday>=5]["amount"].sum())
    if weekend_spend > weekday_spend * 1.3:
        s.append("週末支出顯著高於平日，建議事先規畫行程與預算，避免臨時性大額消費。")

    # 總結性建議（保底三條）
    if len(s) < 3:
        s.append("整體支出尚屬穩定，仍建議維持記帳習慣並設定每週上限，超過即暫停非必要項目。")
    text = " ".join(s)
    # 限制長度（避免過長）
    if len(text) > 260:
        text = text[:260] + "…"
    return "local", text

def write_ai_to_sheets(blocks, provider, text):
    ss = open_spreadsheet()

    # A) 追加到 AI_Analysis 分頁
    ws_ai = ensure_sheet(ss, AI_SHEET_NAME)
    vals = ws_ai.get_all_values()
    if not vals:
        ws_ai.update(values=[["timestamp","week_start","week_end","provider","total_week_amount","top_category","records_count","analysis"]],
                     range_name="A1:H1")

    if blocks["df_week"].empty:
        total_week = 0.0; top_cat = ""; cnt = 0
    else:
        total_week = float(blocks["df_week"]["amount"].sum())
        grp = blocks["df_week"].groupby("category")["amount"].sum().sort_values(ascending=False)
        top_cat = "" if grp.empty else str(grp.index[0])
        cnt = int(len(blocks["df_week"]))

    row = [
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        str(blocks["week_start"]), str(blocks["week_end"]), provider,
        round(total_week,2), top_cat, cnt, text
    ]
    ws_ai.append_row(row)

    # B) 同步附加到 Summary 底部
    ws_sum = ensure_sheet(ss, SUMMARY_SHEET_NAME)
    last = len(ws_sum.get_all_values()) + 2
    ws_sum.update(values=[["=== Weekly Summary (AI / Local) ==="], [text]], range_name=f"A{last}:A{last+1}")

    return "AI 分析已寫入 AI_Analysis 與 Summary ✅"

# 6) Gradio UI（四分頁）
with gr.Blocks(title="HW2：日常支出 + 本地 AI 分析") as demo:
    gr.Markdown("## 📊 HW2：日常支出速算與分攤 ＋ 🧠 本地分析（不連外）")

    # ① 新增一筆消費（寫入）
    with gr.Tab("① 新增一筆消費（寫入）"):
        date_in  = gr.Textbox(label="date (YYYY-MM-DD)")
        cat_in   = gr.Textbox(label="category（餐飲/交通/娛樂…）")
        item_in  = gr.Textbox(label="item")
        amt_in   = gr.Textbox(label="amount（數字）")
        payer_in = gr.Textbox(label="payer（付款人）")
        add_btn  = gr.Button("送出")
        add_msg  = gr.Markdown()

        def on_add(d, c, i, a, p):
            try:
                ss = open_spreadsheet()
                ws = get_ws(ss, DATA_SHEET_NAME)
                if ws is None:
                    ws = ss.add_worksheet(title=DATA_SHEET_NAME, rows=200, cols=10)
                    ws.update(values=[["date","category","item","amount","payer"]], range_name="A1:E1")
                pd.to_datetime(d)
                amt = float(str(a).replace(",","").strip())
                ws.append_row([pd.to_datetime(d).strftime("%Y-%m-%d"), c, i, amt, p])
                return "已新增一筆紀錄 ✅"
            except Exception as e:
                return f"❌ 錯誤：{debug_text(e)}"
        add_btn.click(on_add, inputs=[date_in, cat_in, item_in, amt_in, payer_in], outputs=add_msg)

    # ② 重算並寫回 Summary
    with gr.Tab("② 重算並寫回 Summary（讀→算→寫）"):
        run_btn  = gr.Button("執行")
        out_msg  = gr.Markdown()
        df_view  = gr.Dataframe(label="消費紀錄（預覽）", wrap=True)
        cat_view = gr.Dataframe(label="分類小計")
        aa_view  = gr.Dataframe(label="AA 分攤")
        tr_view  = gr.Dataframe(label="轉帳建議")
        weekly   = gr.Markdown(label="週摘要（非 AI）")

        def on_run():
            try:
                _, df = load_data()
                blocks = compute_summary(df)
                msg = write_summary(blocks)
                return (msg, df, blocks["by_category"], blocks["aa_summary"], blocks["transfer_df"], f"**{blocks['weekly_text']}**")
            except Exception as e:
                return (f"❌ 錯誤：{debug_text(e)}", pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), "")
        run_btn.click(on_run, outputs=[out_msg, df_view, cat_view, aa_view, tr_view, weekly])

    # ③ 只看週摘要（不寫回）
    with gr.Tab("③ 只看週摘要（不寫回）"):
        peek_btn = gr.Button("計算週摘要")
        peek_md  = gr.Markdown()
        def on_peek():
            try:
                _, df = load_data()
                blocks = compute_summary(df)
                return f"**{blocks['weekly_text']}**"
            except Exception as e:
                return f"❌ 錯誤：{debug_text(e)}"
        peek_btn.click(on_peek, outputs=peek_md)

    # ④ 生成「本地 AI」分析並寫回
    with gr.Tab("④ 生成 AI 分析（本地 / 寫入 AI_Analysis + Summary）"):
        ai_btn = gr.Button("生成 本地 AI 分析 → 寫回")
        ai_md  = gr.Markdown()

        def on_ai_generator():
            try:
                yield "▶️ 讀取資料中…"
                _, df = load_data()

                yield "▶️ 計算統計中…"
                blocks = compute_summary(df)

                if blocks["df_week"].empty:
                    yield f"⚠️ 本週（{blocks['week_start']}~{blocks['week_end']}）沒有消費紀錄，請先新增本週資料。"
                    return

                yield "▶️ 產生本地分析…"
                provider, text = local_ai_analysis(blocks["df_week"], blocks["week_start"], blocks["week_end"])

                yield "▶️ 寫回 Google Sheet…"
                msg = write_ai_to_sheets(blocks, provider, text)

                yield f"✅ {msg}\n\n---\n**分析（{provider}）**：\n\n{text}"
            except Exception as e:
                # 也寫回 Sheet，確保有紀錄
                try:
                    _, df = load_data()
                    blocks = compute_summary(df)
                    _ = write_ai_to_sheets(blocks, "error", f"❌ {debug_text(e)}")
                except:
                    pass
                yield f"❌ 錯誤：{debug_text(e)}"

        ai_btn.click(on_ai_generator, outputs=ai_md)

# 啟動
demo.launch(share=True, show_error=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4de5d4dbc1a3efa42b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
